In [1]:
from pathlib import Path
from tempfile import TemporaryDirectory

import torch
from loguru import logger
from mido import MidiFile

from yoshimidi import player
from yoshimidi.data.parse import midi_parsing, one_hot_parsing, token_parsing, track_parsing

pygame 2.5.0 (SDL 2.28.0, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [22]:
PATH_INDEX = 100
midi_path = list(Path("../out/dataset/01_raw/lmd_full/").rglob("*.mid"))[PATH_INDEX]

In [23]:
player.play(midi_path)

2023-08-26 21:30:14.552 | INFO     | yoshimidi.player:play:12 - Playing ../out/dataset/01_raw/lmd_full/9/901ed509cb9b8315ed04c728f9ce5ec2.mid


2023-08-26 21:30:18.987 | INFO     | yoshimidi.player:play:19 - Finishing due to interrupt


In [24]:
CHANNEL_INDEX = 0
midi_file = MidiFile(midi_path)
channels = set(msg.channel for msg in midi_file if hasattr(msg, "channel"))
logger.info("num channels: {}".format(len(channels)))
ym_track = track_parsing.from_midi(midi_file)
assert ym_track is not None
ym_tokens = [token_parsing.from_channel(channel) for channel in ym_track.channels.values()]
ym_tokens = [ym_tokens[CHANNEL_INDEX]]
ym_one_hot = one_hot_parsing.from_tokens(ym_tokens[0], device=torch.device("cpu"), dtype=torch.float32)

ym_track_recons = track_parsing.from_tokens(ym_tokens)
midi_file_recons = midi_parsing.from_tracks([ym_track_recons])

with TemporaryDirectory() as temp:
    path = Path(temp) / "recons.mid"
    midi_file_recons.save(path)
    player.play(path)

2023-08-26 21:30:20.053 | INFO     | __main__:<module>:4 - num channels: 5
2023-08-26 21:30:20.092 | INFO     | yoshimidi.player:play:12 - Playing /var/folders/7w/66fh_d3s5hb0br9f7wtqww1h0000gn/T/tmp2aga9nu3/recons.mid
2023-08-26 21:30:23.366 | INFO     | yoshimidi.player:play:19 - Finishing due to interrupt
